In [6]:
import pandas as pd
import requests

In [7]:
# init watcher. 
# WARNING! Actualise api_key every 24h
api_key = 'RGAPI-d4d343b1-7f49-42fd-a9e4-bf0d4063d903'

In [8]:
regions = [
    'br1', 
    'eun1', 
    'euw1', 
    'jp1', 
    'kr', 
    'la1', 
    'la2', 
    'na1', 
    'oc1', 
    'ph2', 
    'ru', 
    'sg2', 
    'th2', 
    'tr1', 
    'tw2', 
    'vn2'
]

In [9]:
df = pd.DataFrame(columns=[
                            'puuid', 'region', 'tier', 'placement', 'pair_id',
                            'champion_name', 'champion_rarity', 'champion_tier', 
                            'item_1', 'item_2', 'item_3', 
                            'augment_1', 'augment_2', 'augment_3'
                        ])

In [29]:
def accumulate_region_data(api_key, region, tier):
    players = []
    result = requests.get(f'https://{region}.api.riotgames.com/tft/league/v1/{tier}?queue=RANKED_TFT&page=1', headers={"X-Riot-Token": api_key})
    while result.status_code != 200:
        result = requests.get(f'https://{region}.api.riotgames.com/tft/league/v1/{tier}?queue=RANKED_TFT&page=1', headers={"X-Riot-Token": api_key})
        
    for player in result.json()['entries']:
        player_info = {
            'summonerId': player['summonerId'],
            'summonerName': player['summonerName'],
            'region': region,  # Sprawdzamy, czy region jest poprawnie przypisany
            'tier': tier
        }
        players.append(player_info)
    return players

In [30]:
def add_puuid(api_key, players, region):
    for index, player in enumerate(players):
        result = requests.get(f'https://{player["region"]}.api.riotgames.com/tft/league/v1/entries/by-summoner/{player["summonerId"]}', headers={"X-Riot-Token": api_key})
        while result.status_code != 200:
            result = requests.get(f'https://{player["region"]}.api.riotgames.com/tft/league/v1/entries/by-summoner/{player["summonerId"]}', headers={"X-Riot-Token": api_key})
        players[index]['puuid'] = result.json()[0]['puuid']
    return players

In [31]:
import os

In [35]:
def save_matches(api_key, players, bigger_region, region, tier):
    for player in players:
        result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{player["puuid"]}/ids?start=0&count=20', headers={"X-Riot-Token": api_key})
        while result.status_code != 200: 
            result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{player["puuid"]}/ids?start=0&count=20', headers={"X-Riot-Token": api_key})

        
        for match in result.json():
            if f'{match}.csv' in os.listdir('04.04.2024'):
                continue
            
            result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/{match}', headers={"X-Riot-Token": api_key})
            while result.status_code != 200:
                result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/{match}', headers={"X-Riot-Token": api_key})

            if result.json()['info']['tft_game_type'] != 'standard' or result.json()['info']['tft_set_core_name'] !=  'TFTSet11':
                continue

            df = pd.DataFrame(columns=[
                            'puuid', 'bigger_region', 'region', 'tier', 'placement',
                            'champion_name', 'champion_rarity', 'champion_tier', 
                            'item_1', 'item_2', 'item_3', 
                            'augment_1', 'augment_2', 'augment_3'
                        ])

            
            for participant in result.json()['info']['participants']:
                while len(participant['augments']) < 3:
                    participant['augments'].append(pd.NA)
                for unit in participant['units']:
                    while len(unit['itemNames']) < 3:
                        unit['itemNames'].append(pd.NA)

                    df.loc[df.shape[0]] = [
                                            participant['puuid'], bigger_region, region, tier, participant['placement'], 
                                            unit['character_id'], unit['rarity'], unit['tier'],
                                            unit['itemNames'][0], unit['itemNames'][1], unit['itemNames'][2],
                                            participant['augments'][0], participant['augments'][1], participant['augments'][2]
                                          ]

            df.to_csv(f'04.04.2024/{match}.csv')
            print(f'{match}.csv is saved')
            print('=====================')

In [36]:
def get_tft_data(api_key, region, tier='challenger'):
    players = accumulate_region_data(api_key, region, tier)
    print(f'Accumulation of players in {tier} from {region} is done')
    players = add_puuid(api_key, players, region)
    print(f'Adding puuid to players is done.')
    regions_dict = {
                        'americas': ['br1', 'la1', 'la2', 'na1'],
                        'europe': ['eun1', 'euw1', 'ru'],
                        'asia': ['jp1', 'kr', 'ph2', 'sg2', 'th2', 'tr1', 'tw2', 'vn2'],
                        'sea': ['oc1']
                    }

    for key, value in zip(regions_dict.keys(), regions_dict.values()):
        if region in value:
            bigger_region = key
            save_matches(api_key, players, bigger_region, region, tier)

    print('Done')
    
    

In [ ]:
for tier in ['master']:
    for region in regions:
        get_tft_data(api_key, region, tier)

Accumulation of players in master from br1 is done
Adding puuid to players is done.
BR1_2918494648.csv is saved
BR1_2918479408.csv is saved
BR1_2918455197.csv is saved
BR1_2917504335.csv is saved
BR1_2917247474.csv is saved
BR1_2917168264.csv is saved
BR1_2917150186.csv is saved
BR1_2917133137.csv is saved
BR1_2917101803.csv is saved
BR1_2917086967.csv is saved
BR1_2917076920.csv is saved
BR1_2916864805.csv is saved
BR1_2916832028.csv is saved
BR1_2916749872.csv is saved
BR1_2915356175.csv is saved
BR1_2918521038.csv is saved
BR1_2918502876.csv is saved
BR1_2918491068.csv is saved
BR1_2918473277.csv is saved
BR1_2918280168.csv is saved
BR1_2918043669.csv is saved
BR1_2918006582.csv is saved
BR1_2917962499.csv is saved
BR1_2917878236.csv is saved
BR1_2917864460.csv is saved
BR1_2917810788.csv is saved
BR1_2917745767.csv is saved
BR1_2917711536.csv is saved
BR1_2917692345.csv is saved
BR1_2917641756.csv is saved
BR1_2917629619.csv is saved
BR1_2917615424.csv is saved
BR1_2918363572.csv i